# Importing Multielectrode Data


The data file used in this tutorial is freely available from [the companion website](https://www.elsevier.com/books-and-journals/book-companion/9780128040430/chapter-files#Chapter%20Files) for the book,  Nylen, E.L., and Wallisch, P.. (2017). [*Neural Data Science*](https://www.sciencedirect.com/book/9780128040430/neural-data-science). Academic Press. 

The code provided here, along with the explanatory test, was written by Aaron J. Newman. 

## Import packages

In [1]:
import scipy.io
from collections import defaultdict
import numpy as np

## Set variables

We hard-code some known properties of the experiment so that we can easily use these values later. It's good practice only to code those things that you know you can't read from the data. For example, we should be able to derive the number of trials, number of channels (electrodes), etc. from the data, so we won't hard-code those. This makes the code more robust — for example if we had data from other animals that used a microarray with a different number of channels, or ran fewer trials, etc..


In [2]:
noise_codes = [0, 255] # explained later

# time_base is used to define x axis in plots over time. We know that the trials went from 
#   150 ms pre-stimulus to 2000 ms post-stimulus onset. Here we specify a wider range, 
#   from -200 to 2500 ms (in 10 ms increments), to provide some padding.
time_base = np.arange(-.2, 2.5, .01) 

# times the stimulus went on and off
grating_on_time  = 0
grating_off_time = 2

## Import the data

The data are stored in a Matlab file, which has the extension `.mat`. The `scipy` package has a function that reads Matlab files into Python. Our first adventure will be deciphering how the data were stored in the Matlab file, which is much more complex than a typical CSV or other tabular-type format.

In [3]:
data_in = scipy.io.loadmat('arrayDATA.mat')

## Data structure
According to the book, DATA contains 2300 rows with two fields (note: "fields" not columns; this is a complex data structure): `nev` and `ori`. Each row represents a trial; `nev` is a matrix for each trial that represents the spiking information, and `ori` is the orientation of the stimulus for each trial (0 or 90 deg). 

Rows in each `nev` matrix represent individual spikes per trial, one for each spike. Column 1 encodes which electrode the spike was recorded on, column 2 is a sort code, and column 3 is the time since the beginning of the trial. So, like the "ten intensities" data in the previous lesson, the data record the times of individual spikes, rather than providing a time series with 1s when spikes occurred, and 0s otherwise.

`Sort code` is a number from 0-255, indicating which neuron a spike was inferred to have come from (based on spike sorting, which was already performed on the data). Sort codes 0 and 255 are for noise (hence our defining `noise_codes` above); all other sort codes are valid neurons. However, in this data set the only other sort codes are the numbers 1-4, which is somewhat puzzling and not explained; one might expect more than 4 neurons. Indeed, for our purposes we will look at the activity, and correlations between, the channels (electrodes) in the microarray rather than using sort codes.

Although all the answers are provided for you here, this section takes you on the typical journey a data scientist might embark upon when working with a new dataset, where the format is not fully documented. Our first question is what type the data is:

In [4]:
type(data_in)

dict

Now let's display the dictionary:

In [5]:
data_in

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Jun 16 15:54:00 2016',
 '__version__': '1.0',
 '__globals__': [],
 'DATA': array([[(array([[ 20.   ,   1.   ,   0.624],
        [ 20.   ,   1.   ,   0.63 ],
        [ 20.   ,   3.   ,   0.652],
        [ 20.   ,   3.   ,   0.659],
        [ 20.   , 255.   ,   0.686],
        [ 20.   ,   1.   ,   0.689],
        [ 20.   ,   1.   ,   0.711],
        [ 20.   , 255.   ,   0.731],
        [ 20.   ,   3.   ,   0.742],
        [ 20.   ,   3.   ,   0.757],
        [ 20.   ,   2.   ,   0.764],
        [ 20.   ,   4.   ,   0.779],
        [ 20.   ,   2.   ,   0.786],
        [ 20.   ,   3.   ,   0.812],
        [ 20.   ,   1.   ,   0.826],
        [ 20.   ,   3.   ,   0.848],
        [ 20.   , 255.   ,   0.857],
        [ 20.   ,   4.   ,   0.866],
        [ 20.   ,   1.   ,   0.872],
        [ 20.   ,   3.   ,   0.883],
        [ 20.   ,   3.   ,   0.898],
        [ 20.   ,   4.   ,   0.909],
        [ 20.   , 255.   ,   0

Whoa, that looks complex! But we can see that the dictionary has only a few keys. The first three are information about the file (meta-data), while the fourth, `DATA`, appears to be our data. We can see that this is a numpy array, which we can confirm by asking its type:

In [6]:
type(data_in['DATA'])

numpy.ndarray

Next let's get the shape of the data array:

In [7]:
data_in['DATA'].shape

(2300, 1)

OK, this makes some sense since we know that the experiment comprised 2300 trials, and its consistent with the documentation in the book that DATA contains 2300 rows. So there is one row per trial in the DATA array, and each row is only one column. However, that one column actually contains an embedded array; note above that the first row of DATA is:

` 'DATA': array([[(array([[ 20.   ,   1.   ,   0.624],`

We know that numpy arrays are represented as square brackets within parentheses, as such: `array([])`. The fact that here we see `array([[(array([[` suggests that we have arrays embedded inside other arrays. It will take some digging to figure this out!

In [8]:
data_in['DATA'][0].shape

(1,)

Just one thing? So what's in that np array?

In [9]:
data_in['DATA'][0]

array([(array([[ 20.   ,   1.   ,   0.624],
       [ 20.   ,   1.   ,   0.63 ],
       [ 20.   ,   3.   ,   0.652],
       [ 20.   ,   3.   ,   0.659],
       [ 20.   , 255.   ,   0.686],
       [ 20.   ,   1.   ,   0.689],
       [ 20.   ,   1.   ,   0.711],
       [ 20.   , 255.   ,   0.731],
       [ 20.   ,   3.   ,   0.742],
       [ 20.   ,   3.   ,   0.757],
       [ 20.   ,   2.   ,   0.764],
       [ 20.   ,   4.   ,   0.779],
       [ 20.   ,   2.   ,   0.786],
       [ 20.   ,   3.   ,   0.812],
       [ 20.   ,   1.   ,   0.826],
       [ 20.   ,   3.   ,   0.848],
       [ 20.   , 255.   ,   0.857],
       [ 20.   ,   4.   ,   0.866],
       [ 20.   ,   1.   ,   0.872],
       [ 20.   ,   3.   ,   0.883],
       [ 20.   ,   3.   ,   0.898],
       [ 20.   ,   4.   ,   0.909],
       [ 20.   , 255.   ,   0.926],
       [ 20.   ,   3.   ,   0.951],
       [ 20.   ,   1.   ,   0.968],
       [ 20.   , 255.   ,   0.981],
       [ 20.   , 255.   ,   0.992],
       [ 20.   , 255

Hmm, `array([(array([[ ` tells us there's a numpy array inside the numpy array. The fact that the embedded array appears as `array([[ ` with two open square brackets tells us that the embedded array itself contains multiple entries. So, what's the first entry in the embedded array? As you can see below, we use a sequence of numbers in square brackets to index increasingly-deep levels of the data structure:

In [10]:
data_in['DATA'][0][0]

(array([[ 20.   ,   1.   ,   0.624],
       [ 20.   ,   1.   ,   0.63 ],
       [ 20.   ,   3.   ,   0.652],
       [ 20.   ,   3.   ,   0.659],
       [ 20.   , 255.   ,   0.686],
       [ 20.   ,   1.   ,   0.689],
       [ 20.   ,   1.   ,   0.711],
       [ 20.   , 255.   ,   0.731],
       [ 20.   ,   3.   ,   0.742],
       [ 20.   ,   3.   ,   0.757],
       [ 20.   ,   2.   ,   0.764],
       [ 20.   ,   4.   ,   0.779],
       [ 20.   ,   2.   ,   0.786],
       [ 20.   ,   3.   ,   0.812],
       [ 20.   ,   1.   ,   0.826],
       [ 20.   ,   3.   ,   0.848],
       [ 20.   , 255.   ,   0.857],
       [ 20.   ,   4.   ,   0.866],
       [ 20.   ,   1.   ,   0.872],
       [ 20.   ,   3.   ,   0.883],
       [ 20.   ,   3.   ,   0.898],
       [ 20.   ,   4.   ,   0.909],
       [ 20.   , 255.   ,   0.926],
       [ 20.   ,   3.   ,   0.951],
       [ 20.   ,   1.   ,   0.968],
       [ 20.   , 255.   ,   0.981],
       [ 20.   , 255.   ,   0.992],
       [ 20.   , 255.   ,  

So, this looks like a long numpy array with 3 columns, and then right down at the bottom of the output, you'll see a second array with a single value (90). That's followed by a final entry that is a `dtype` specification but instead of a normal Python data type, it's a list containing two tuples, one for 'nev' and one for 'ori'. These provide labels for the contents of the two arrays.

Note, however, that the output above starts with `(`, whereas above the output of `data_in['DATA'][0]` started with `array` without a leading `(`. So there's actually another level of embedding. If we ask for the type we see:

In [11]:
type(data_in['DATA'][0][0])

numpy.void

In [12]:
data_in['DATA'][0][0].shape

()

We haven't encountered `numpy.void` types before, but they are a special, generic numpy data type that allows one to create a numpy object of arbitrary size. The details of this are beyond the scope of this lesson, but suffice to say that this is a necessary part of converting Matlab's data structures to numpy format. The thing you do need to know is that this `void` data structure adds one more level of embedding, so we need to add another index to drill down into the data:

In [13]:
type(data_in['DATA'][0][0][0])

numpy.ndarray

In [14]:
data_in['DATA'][0][0][0].shape

(207, 3)

So `data_in['DATA'][0][0][0]` is the `nev` portion of the data for the first trial in the experiment. It's long, but if we look at the first 10 rows we can confirm that this is the `nev` array for this trial, with 3 columns (electrode ID, sort code, and time of spike).
Rows in each `nev` matrix represent individual spikes per trial, one for each spike. Column 1 encodes which electrode the spike was recorded on, column 2 is a sort code, and column 3 is the time since the beginning of the trial. 

In [15]:
data_in['DATA'][0][0][0][:10]

array([[ 20.   ,   1.   ,   0.624],
       [ 20.   ,   1.   ,   0.63 ],
       [ 20.   ,   3.   ,   0.652],
       [ 20.   ,   3.   ,   0.659],
       [ 20.   , 255.   ,   0.686],
       [ 20.   ,   1.   ,   0.689],
       [ 20.   ,   1.   ,   0.711],
       [ 20.   , 255.   ,   0.731],
       [ 20.   ,   3.   ,   0.742],
       [ 20.   ,   3.   ,   0.757]], dtype=float32)

Each trial's data also contains the orientation of the stimulus, in the second field of the embedded NumPy array. Let's look at this for the first trial:

In [16]:
data_in['DATA'][0][0][1]

array([[90.]], dtype=float32)

Note that the value (90) is sinde two sets of brackets, so to get the orientation as a simple float, we need to go down two more embeddings:

In [17]:
data_in['DATA'][0][0][1][0][0]

90.0

OK, now let's look at the second trial:

In [18]:
data_in['DATA'][1][0][0].shape

(1655, 3)

In [19]:
data_in['DATA'][1][0][0]

array([[ 7.800e+01,  0.000e+00, -1.080e-01],
       [ 9.400e+01,  1.000e+00, -1.090e-01],
       [ 3.500e+01,  2.550e+02, -1.050e-01],
       ...,
       [ 1.900e+01,  1.000e+00,  2.614e+00],
       [ 2.200e+01,  1.000e+00,  2.618e+00],
       [ 8.000e+00,  1.000e+00,  2.620e+00]], dtype=float32)

Each trial has a different number of rows (depending on the number of spikes that were recorded on that trial), but always 3 columns. Note that numpy sometimes shows you values in standard float notation (as we saw above for the first trial), and sometimes as scientific (exponential) notation (as we see for this second trial). The latter is used if there are very small or large values that would take too many digits to represent in standard notation. These are just different ways of showing the data; note that for both trials, the `dtype=float32`.

Let's check the orientation for this second trial:

In [20]:
data_in['DATA'][1][0][1][0][0]

0.0